In [1]:
pip install tensorflow torch torchvision scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [2]:
#VGG16
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, Xception, InceptionV3, EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build and compile a model using a pre-trained CNN
def build_model(base_model_fn, preprocess_fn, img_size=IMG_SIZE):
    # Load the pre-trained model
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=img_size + (3,))
    
    # Freeze the pre-trained model layers
    base_model.trainable = False
    
    # Build the model
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
        tf.keras.layers.Lambda(preprocess_fn),  # Preprocessing specific to the model
        base_model,
        GlobalAveragePooling2D(),  # Use Global Average Pooling instead of Flatten for better generalization
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Choose a pre-trained model (you can switch between these models)
model_name = 'VGG16'  # Options: 'VGG16', 'ResNet50', 'Xception', 'InceptionV3', 'EfficientNetB0'

if model_name == 'VGG16':
    model = build_model(VGG16, vgg16_preprocess)
elif model_name == 'ResNet50':
    model = build_model(ResNet50, resnet50_preprocess)
elif model_name == 'Xception':
    model = build_model(Xception, xception_preprocess)
elif model_name == 'InceptionV3':
    model = build_model(InceptionV3, inception_preprocess)
elif model_name == 'EfficientNetB0':
    model = build_model(EfficientNetB0, efficientnet_preprocess)

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(



Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 579s 4s/step - accuracy: 0.5111 - loss: 1.4091 - val_accuracy: 0.6799 - val_loss: 0.7344
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 582s 5s/step - accuracy: 0.7257 - loss: 0.6228 - val_accuracy: 0.7798 - val_loss: 0.5581
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 595s 4s/step - accuracy: 0.8410 - loss: 0.4395 - val_accuracy: 0.8312 - val_loss: 0.4516
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 563s 4s/step - accuracy: 0.8816 - loss: 0.3393 - val_accuracy: 0.8235 - val_loss: 0.4194
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 561s 4s/step - accuracy: 0.9123 - loss: 0.2573 - val_accuracy: 0.9011 - val_loss: 0.2839
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 561s 4s/step - accuracy: 0.9597 - loss: 0.1797 - val_accuracy: 0.9282 - val_loss: 0.2389
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 583s 5s/step - accuracy: 0.9746 - loss: 0.1269 - val_accuracy: 0.9370 - val_loss: 0.2065
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 563s 4s/step - accuracy: 0.9792 - loss: 0.1039 - val_acc

In [3]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 116s 4s/step
              precision    recall  f1-score   support

          AD       0.87      0.98      0.92       246
          CI       0.95      0.96      0.96       503
          CN       0.99      0.86      0.92       282

    accuracy                           0.94      1031
   macro avg       0.94      0.93      0.93      1031
weighted avg       0.94      0.94      0.94      1031



In [4]:
#Resnet50
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, Xception, InceptionV3, EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build and compile a model using a pre-trained CNN
def build_model(base_model_fn, preprocess_fn, img_size=IMG_SIZE):
    # Load the pre-trained model
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=img_size + (3,))
    
    # Freeze the pre-trained model layers
    base_model.trainable = False
    
    # Build the model
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
        tf.keras.layers.Lambda(preprocess_fn),  # Preprocessing specific to the model
        base_model,
        GlobalAveragePooling2D(),  # Use Global Average Pooling instead of Flatten for better generalization
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Choose a pre-trained model (you can switch between these models)
model_name = 'ResNet50'  # Options: 'VGG16', 'ResNet50', 'Xception', 'InceptionV3', 'EfficientNetB0'

if model_name == 'VGG16':
    model = build_model(VGG16, vgg16_preprocess)
elif model_name == 'ResNet50':
    model = build_model(ResNet50, resnet50_preprocess)
elif model_name == 'Xception':
    model = build_model(Xception, xception_preprocess)
elif model_name == 'InceptionV3':
    model = build_model(InceptionV3, inception_preprocess)
elif model_name == 'EfficientNetB0':
    model = build_model(EfficientNetB0, efficientnet_preprocess)

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 222s 2s/step - accuracy: 0.4766 - loss: 1.2541 - val_accuracy: 0.5626 - val_loss: 0.8698
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 196s 2s/step - accuracy: 0.6732 - loss: 0.7762 - val_accuracy: 0.7032 - val_loss: 0.7182
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.7423 - loss: 0.6631 - val_accuracy: 0.7565 - val_loss: 0.6113
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 190s 1s/step - accuracy: 0.7728 - loss: 0.5723 - val_accuracy: 0.7546 - val_loss: 0.6070
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 198s 1s/step - accuracy: 0.8099 - loss: 0.4907 - val_accuracy: 0.7983 - val_loss: 0.5353
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 193s 1s/step - accuracy: 0.8457 - loss: 0.4314 - val_accuracy: 0.8516 - val_loss: 0.4160
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.8910 - loss: 0.3376 - val_accuracy: 0.8235 - val_loss: 0.4189
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 187s 1s/step - accuracy: 0.8986 - loss: 0.3107 - val_accu

In [5]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step
              precision    recall  f1-score   support

          AD       0.98      0.78      0.87       228
          CI       0.86      0.99      0.92       515
          CN       0.91      0.81      0.86       288

    accuracy                           0.89      1031
   macro avg       0.91      0.86      0.88      1031
weighted avg       0.90      0.89      0.89      1031



In [6]:
#Xception
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, Xception, InceptionV3, EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build and compile a model using a pre-trained CNN
def build_model(base_model_fn, preprocess_fn, img_size=IMG_SIZE):
    # Load the pre-trained model
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=img_size + (3,))
    
    # Freeze the pre-trained model layers
    base_model.trainable = False
    
    # Build the model
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
        tf.keras.layers.Lambda(preprocess_fn),  # Preprocessing specific to the model
        base_model,
        GlobalAveragePooling2D(),  # Use Global Average Pooling instead of Flatten for better generalization
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Choose a pre-trained model (you can switch between these models)
model_name = 'Xception'  # Options: 'VGG16', 'ResNet50', 'Xception', 'InceptionV3', 'EfficientNetB0'

if model_name == 'VGG16':
    model = build_model(VGG16, vgg16_preprocess)
elif model_name == 'ResNet50':
    model = build_model(ResNet50, resnet50_preprocess)
elif model_name == 'Xception':
    model = build_model(Xception, xception_preprocess)
elif model_name == 'InceptionV3':
    model = build_model(InceptionV3, inception_preprocess)
elif model_name == 'EfficientNetB0':
    model = build_model(EfficientNetB0, efficientnet_preprocess)

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.4863 - loss: 1.0502 - val_accuracy: 0.5742 - val_loss: 0.8899
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 288s 2s/step - accuracy: 0.6151 - loss: 0.8500 - val_accuracy: 0.6596 - val_loss: 0.7975
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 292s 2s/step - accuracy: 0.6674 - loss: 0.7512 - val_accuracy: 0.6770 - val_loss: 0.7486
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 283s 2s/step - accuracy: 0.7015 - loss: 0.6993 - val_accuracy: 0.7032 - val_loss: 0.6959
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 255s 2s/step - accuracy: 0.7321 - loss: 0.6517 - val_accuracy: 0.7468 - val_loss: 0.6337
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 254s 2s/step - accuracy: 0.7651 - loss: 0.5905 - val_accuracy: 0.7371 - val_loss: 0.6261
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 249s 2s/step - accuracy: 0.7703 - loss: 0.5611 - val_accuracy: 0.7711 - val_loss: 0.5498
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 252s 2s/step - accuracy: 0.8101 - loss: 0.5003 - val_accu

In [7]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 52s 2s/step
              precision    recall  f1-score   support

          AD       0.87      0.55      0.67       210
          CI       0.80      0.93      0.86       515
          CN       0.82      0.81      0.82       306

    accuracy                           0.82      1031
   macro avg       0.83      0.76      0.78      1031
weighted avg       0.82      0.82      0.81      1031



In [8]:
#EfficientNetB0
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, Xception, InceptionV3, EfficientNetB0
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build and compile a model using a pre-trained CNN
def build_model(base_model_fn, preprocess_fn, img_size=IMG_SIZE):
    # Load the pre-trained model
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=img_size + (3,))
    
    # Freeze the pre-trained model layers
    base_model.trainable = False
    
    # Build the model
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
        tf.keras.layers.Lambda(preprocess_fn),  # Preprocessing specific to the model
        base_model,
        GlobalAveragePooling2D(),  # Use Global Average Pooling instead of Flatten for better generalization
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Choose a pre-trained model (you can switch between these models)
model_name = 'EfficientNetB0'  # Options: 'VGG16', 'ResNet50', 'Xception', 'InceptionV3', 'EfficientNetB0'

if model_name == 'VGG16':
    model = build_model(VGG16, vgg16_preprocess)
elif model_name == 'ResNet50':
    model = build_model(ResNet50, resnet50_preprocess)
elif model_name == 'Xception':
    model = build_model(Xception, xception_preprocess)
elif model_name == 'InceptionV3':
    model = build_model(InceptionV3, inception_preprocess)
elif model_name == 'EfficientNetB0':
    model = build_model(EfficientNetB0, efficientnet_preprocess)

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 66s 434ms/step - accuracy: 0.5247 - loss: 0.9771 - val_accuracy: 0.6159 - val_loss: 0.8177
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 66s 510ms/step - accuracy: 0.6594 - loss: 0.7800 - val_accuracy: 0.7236 - val_loss: 0.7114
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 63s 488ms/step - accuracy: 0.7039 - loss: 0.6939 - val_accuracy: 0.7565 - val_loss: 0.6439
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 65s 508ms/step - accuracy: 0.7573 - loss: 0.6155 - val_accuracy: 0.7905 - val_loss: 0.5728
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 65s 501ms/step - accuracy: 0.7786 - loss: 0.5559 - val_accuracy: 0.7420 - val_loss: 0.5768
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 64s 498ms/step - accuracy: 0.7938 - loss: 0.5111 - val_accuracy: 0.8361 - val_loss: 0.4646
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 64s 493ms/step - accuracy: 0.8150 - loss: 0.4665 - val_accuracy: 0.8109 - val_loss: 0.4667
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 64s 499ms/step - accuracy: 0.8423 - loss: 0

In [9]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 16s 453ms/step
              precision    recall  f1-score   support

          AD       0.81      0.89      0.85       214
          CI       0.90      0.95      0.92       518
          CN       0.95      0.79      0.86       299

    accuracy                           0.89      1031
   macro avg       0.89      0.88      0.88      1031
weighted avg       0.90      0.89      0.89      1031



In [10]:
pip install git+https://github.com/rcmalli/keras-squeezenet.git

  Cloning https://github.com/rcmalli/keras-squeezenet.git to c:\users\admin\appdata\local\temp\pip-req-build-ryxat59p
Note: you may need to restart the kernel to use updated packages.


  ERROR: Error [WinError 2] The system cannot find the file specified while executing command git version
ERROR: Cannot find command 'git' - do you have 'git' installed and in your PATH?


In [11]:
pip install keras-squeezenet

Note: you may need to restart the kernel to use updated packages.


In [12]:
#Alexnet
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build AlexNet model
def build_alexnet(input_shape, num_classes):
    model = Sequential([
        Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to build a simple SqueezeNet-like model
def build_squeezenet(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3)),
        
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3)),
        
        Conv2D(256, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3)),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Choose a model (you can switch between these models)
model_name = 'AlexNet'  # Options: 'AlexNet', 'SqueezeNet', 'InceptionV3' (for GoogLeNet approximation)

if model_name == 'AlexNet':
    model = build_alexnet(IMG_SIZE + (3,), y_train.shape[1])
elif model_name == 'SqueezeNet':
    model = build_squeezenet(IMG_SIZE + (3,), y_train.shape[1])
elif model_name == 'InceptionV3':  # Approximation for GoogLeNet
    from tensorflow.keras.applications import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=IMG_SIZE + (3,)),
        tf.keras.layers.Lambda(inception_preprocess),
        InceptionV3(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,)),
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 119s 909ms/step - accuracy: 0.4632 - loss: 35.2224 - val_accuracy: 0.4879 - val_loss: 1.0439
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 117s 908ms/step - accuracy: 0.5137 - loss: 1.0318 - val_accuracy: 0.4879 - val_loss: 1.0521
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 114s 886ms/step - accuracy: 0.5067 - loss: 1.0355 - val_accuracy: 0.4879 - val_loss: 1.0411
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 114s 885ms/step - accuracy: 0.5089 - loss: 1.0327 - val_accuracy: 0.4879 - val_loss: 1.0410
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 115s 890ms/step - accuracy: 0.5070 - loss: 1.0338 - val_accuracy: 0.4879 - val_loss: 1.0414
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 114s 886ms/step - accuracy: 0.5188 - loss: 1.0260 - val_accuracy: 0.4879 - val_loss: 1.0409
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 115s 895ms/step - accuracy: 0.5054 - loss: 1.0345 - val_accuracy: 0.4879 - val_loss: 1.0439
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 115s 891ms/step - accuracy: 0.5167 

In [13]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 9s 266ms/step
              precision    recall  f1-score   support

          AD       0.00      0.00      0.00       220
          CI       0.49      1.00      0.66       503
          CN       0.00      0.00      0.00       308

    accuracy                           0.49      1031
   macro avg       0.16      0.33      0.22      1031
weighted avg       0.24      0.49      0.32      1031



C:\Users\admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\admin\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [14]:
#Googlenet
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build AlexNet model
def build_alexnet(input_shape, num_classes):
    model = Sequential([
        Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to build a simple SqueezeNet-like model
def build_squeezenet(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3)),
        
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3)),
        
        Conv2D(256, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3)),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Choose a model (you can switch between these models)
model_name = 'InceptionV3'  # Options: 'AlexNet', 'SqueezeNet', 'InceptionV3' (for GoogLeNet approximation)

if model_name == 'AlexNet':
    model = build_alexnet(IMG_SIZE + (3,), y_train.shape[1])
elif model_name == 'SqueezeNet':
    model = build_squeezenet(IMG_SIZE + (3,), y_train.shape[1])
elif model_name == 'InceptionV3':  # Approximation for GoogLeNet
    from tensorflow.keras.applications import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=IMG_SIZE + (3,)),
        tf.keras.layers.Lambda(inception_preprocess),
        InceptionV3(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,)),
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")

Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 414s 3s/step - accuracy: 0.6047 - loss: 0.8715 - val_accuracy: 0.5655 - val_loss: 3.4789
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 371s 3s/step - accuracy: 0.9104 - loss: 0.2582 - val_accuracy: 0.3317 - val_loss: 5.2078
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.9464 - loss: 0.1511 - val_accuracy: 0.7682 - val_loss: 1.1675
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.9661 - loss: 0.0987 - val_accuracy: 0.9331 - val_loss: 0.2088
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 368s 3s/step - accuracy: 0.9792 - loss: 0.0658 - val_accuracy: 0.7294 - val_loss: 1.4509
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 370s 3s/step - accuracy: 0.9732 - loss: 0.0855 - val_accuracy: 0.6596 - val_loss: 2.0211
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 369s 3s/step - accuracy: 0.9728 - loss: 0.0874 - val_accuracy: 0.9738 - val_loss: 0.0669
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 375s 3s/step - accuracy: 0.9882 - loss: 0.0353 - val_accu

In [15]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 34s 968ms/step
              precision    recall  f1-score   support

          AD       0.64      1.00      0.78       218
          CI       1.00      0.82      0.90       521
          CN       0.97      0.86      0.91       292

    accuracy                           0.87      1031
   macro avg       0.87      0.89      0.86      1031
weighted avg       0.91      0.87      0.88      1031



In [16]:
#Squeezenet
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build AlexNet model
def build_alexnet(input_shape, num_classes):
    model = Sequential([
        Conv2D(96, kernel_size=(11, 11), strides=4, activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Conv2D(256, kernel_size=(5, 5), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(384, kernel_size=(3, 3), padding='same', activation='relu'),
        Conv2D(256, kernel_size=(3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(3, 3), strides=2),
        
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to build a simple SqueezeNet-like model
def build_squeezenet(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(3, 3)),
        
        Conv2D(128, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3)),
        
        Conv2D(256, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(3, 3)),
        
        Flatten(),
        Dense(512, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Choose a model (you can switch between these models)
model_name = 'SqueezeNet'  # Options: 'AlexNet', 'SqueezeNet', 'InceptionV3' (for GoogLeNet approximation)

if model_name == 'AlexNet':
    model = build_alexnet(IMG_SIZE + (3,), y_train.shape[1])
elif model_name == 'SqueezeNet':
    model = build_squeezenet(IMG_SIZE + (3,), y_train.shape[1])
elif model_name == 'InceptionV3':  # Approximation for GoogLeNet
    from tensorflow.keras.applications import InceptionV3
    from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=IMG_SIZE + (3,)),
        tf.keras.layers.Lambda(inception_preprocess),
        InceptionV3(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,)),
        GlobalAveragePooling2D(),
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 107s 820ms/step - accuracy: 0.4989 - loss: 23.2016 - val_accuracy: 0.6557 - val_loss: 0.7640
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 111s 864ms/step - accuracy: 0.7805 - loss: 0.5726 - val_accuracy: 0.9030 - val_loss: 0.2594
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 113s 877ms/step - accuracy: 0.9086 - loss: 0.2342 - val_accuracy: 0.9350 - val_loss: 0.1945
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 112s 869ms/step - accuracy: 0.9642 - loss: 0.1094 - val_accuracy: 0.9447 - val_loss: 0.1665
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 113s 875ms/step - accuracy: 0.9681 - loss: 0.0967 - val_accuracy: 0.9942 - val_loss: 0.0365
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 111s 857ms/step - accuracy: 0.9844 - loss: 0.0466 - val_accuracy: 0.9922 - val_loss: 0.0287
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 113s 876ms/step - accuracy: 0.9860 - loss: 0.0379 - val_accuracy: 0.9864 - val_loss: 0.0627
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 119s 918ms/step - accuracy: 0.9971 

In [17]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 8s 228ms/step
              precision    recall  f1-score   support

          AD       0.94      0.99      0.96       227
          CI       1.00      0.95      0.97       509
          CN       0.96      0.99      0.97       295

    accuracy                           0.97      1031
   macro avg       0.96      0.98      0.97      1031
weighted avg       0.97      0.97      0.97      1031



In [18]:
#VGG16+LSTM
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, GlobalAveragePooling2D, TimeDistributed, Flatten
from tensorflow.keras.layers import Reshape

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build and compile a model with LSTM or BiLSTM
def build_model_lstm(base_model_fn, preprocess_fn, lstm_type='LSTM', img_size=IMG_SIZE, timesteps=1):
    # Load the pre-trained model (e.g., VGG16)
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=img_size + (3,))
    
    # Freeze the pre-trained model layers
    base_model.trainable = False
    
    # Build the model
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
        tf.keras.layers.Lambda(preprocess_fn),  # Preprocessing specific to the model
        base_model,
        GlobalAveragePooling2D(),  # Extract features from the CNN
        Reshape((timesteps, -1)),  # Reshape to (timesteps, features) for LSTM input
        
        # LSTM or BiLSTM layer based on the choice
        LSTM(128, return_sequences=False) if lstm_type == 'LSTM' else Bidirectional(LSTM(128, return_sequences=False)),
        
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Choose a pre-trained model (using VGG16 for feature extraction)
model_name = 'VGG16'

# Choose LSTM or BiLSTM
lstm_type = 'LSTM'  # Options: 'LSTM', 'BiLSTM'

if model_name == 'VGG16':
    if lstm_type == 'LSTM':
        model = build_model_lstm(VGG16, vgg16_preprocess, lstm_type='LSTM')
    elif lstm_type == 'BiLSTM':
        model = build_model_lstm(VGG16, vgg16_preprocess, lstm_type='BiLSTM')

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 664s 5s/step - accuracy: 0.5445 - loss: 0.9523 - val_accuracy: 0.6857 - val_loss: 0.6832
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 637s 5s/step - accuracy: 0.7812 - loss: 0.5737 - val_accuracy: 0.7420 - val_loss: 0.6287
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 639s 5s/step - accuracy: 0.8565 - loss: 0.3788 - val_accuracy: 0.8836 - val_loss: 0.3031
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 495s 4s/step - accuracy: 0.9256 - loss: 0.2087 - val_accuracy: 0.9176 - val_loss: 0.2058
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 469s 4s/step - accuracy: 0.9611 - loss: 0.1271 - val_accuracy: 0.9273 - val_loss: 0.1987
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 463s 4s/step - accuracy: 0.9850 - loss: 0.0634 - val_accuracy: 0.9418 - val_loss: 0.1494
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 471s 4s/step - accuracy: 0.9710 - loss: 0.0859 - val_accuracy: 0.9631 - val_loss: 0.0981
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 459s 4s/step - accuracy: 0.9955 - loss: 0.0275 - val_accu

In [19]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 96s 3s/step
              precision    recall  f1-score   support

          AD       1.00      0.91      0.95       230
          CI       0.97      0.99      0.98       525
          CN       0.95      0.99      0.97       276

    accuracy                           0.97      1031
   macro avg       0.97      0.96      0.97      1031
weighted avg       0.97      0.97      0.97      1031



In [20]:
#VGG16+Bilstm
import os
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, GlobalAveragePooling2D, TimeDistributed, Flatten
from tensorflow.keras.layers import Reshape

# Set the dataset directory
dataset_dir = r'D:\MuTTu\research\AZD\3class AD\dataset\dataset'

# Image size for CNN models
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 10

# Load dataset manually and split it into train/validation sets
def load_and_split_dataset(dataset_dir, test_size=0.2):
    datagen = ImageDataGenerator(preprocessing_function=None)
    data_generator = datagen.flow_from_directory(
        dataset_dir,
        target_size=IMG_SIZE,  # Image size as per the pre-trained model
        batch_size=1,  # Load one image at a time
        class_mode='categorical',
        shuffle=True)
    
    images, labels = [], []
    
    # Loop over the entire dataset and load images one by one
    for i in range(len(data_generator)):
        img, lbl = next(data_generator)  # Using next() function to get image and label
        images.append(img[0])  # Append the first (and only) image in the batch
        labels.append(lbl[0])  # Append the first (and only) label in the batch

    images = np.array(images)
    labels = np.array(labels)

    # Split dataset into training and validation sets
    X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=test_size, random_state=42)
    
    # Get class names
    class_names = list(data_generator.class_indices.keys())
    
    # Return training data, validation data, and class names
    return X_train, X_val, y_train, y_val, class_names

# Load and split dataset
X_train, X_val, y_train, y_val, class_names = load_and_split_dataset(dataset_dir)

# Ensure class_names is correctly returned and stored
print(class_names)  # This will print the class names to verify

# Function to build and compile a model with LSTM or BiLSTM
def build_model_lstm(base_model_fn, preprocess_fn, lstm_type='LSTM', img_size=IMG_SIZE, timesteps=1):
    # Load the pre-trained model (e.g., VGG16)
    base_model = base_model_fn(weights='imagenet', include_top=False, input_shape=img_size + (3,))
    
    # Freeze the pre-trained model layers
    base_model.trainable = False
    
    # Build the model
    model = Sequential([
        tf.keras.layers.InputLayer(input_shape=img_size + (3,)),
        tf.keras.layers.Lambda(preprocess_fn),  # Preprocessing specific to the model
        base_model,
        GlobalAveragePooling2D(),  # Extract features from the CNN
        Reshape((timesteps, -1)),  # Reshape to (timesteps, features) for LSTM input
        
        # LSTM or BiLSTM layer based on the choice
        LSTM(128, return_sequences=False) if lstm_type == 'LSTM' else Bidirectional(LSTM(128, return_sequences=False)),
        
        Dense(128, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')  # Output layer for multi-class classification
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Choose a pre-trained model (using VGG16 for feature extraction)
model_name = 'VGG16'

# Choose LSTM or BiLSTM
lstm_type = 'BiLSTM'  # Options: 'LSTM', 'BiLSTM'

if model_name == 'VGG16':
    if lstm_type == 'LSTM':
        model = build_model_lstm(VGG16, vgg16_preprocess, lstm_type='LSTM')
    elif lstm_type == 'BiLSTM':
        model = build_model_lstm(VGG16, vgg16_preprocess, lstm_type='BiLSTM')

# Train the model
history = model.fit(X_train, y_train, 
                    epochs=EPOCHS, 
                    batch_size=BATCH_SIZE, 
                    validation_data=(X_val, y_val))

# Evaluate the model
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"Validation Accuracy: {val_acc * 100:.2f}%")


Found 5154 images belonging to 3 classes.
['AD', 'CI', 'CN']


C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 489s 4s/step - accuracy: 0.5319 - loss: 0.9574 - val_accuracy: 0.7488 - val_loss: 0.6539
Epoch 2/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 453s 4s/step - accuracy: 0.7869 - loss: 0.5555 - val_accuracy: 0.8371 - val_loss: 0.4129
Epoch 3/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 446s 3s/step - accuracy: 0.9096 - loss: 0.2855 - val_accuracy: 0.9214 - val_loss: 0.2265
Epoch 4/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 498s 4s/step - accuracy: 0.9451 - loss: 0.1616 - val_accuracy: 0.9612 - val_loss: 0.1358
Epoch 5/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 504s 4s/step - accuracy: 0.9777 - loss: 0.0828 - val_accuracy: 0.9661 - val_loss: 0.0948
Epoch 6/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 508s 4s/step - accuracy: 0.9839 - loss: 0.0570 - val_accuracy: 0.9428 - val_loss: 0.1282
Epoch 7/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 472s 4s/step - accuracy: 0.9806 - loss: 0.0665 - val_accuracy: 0.9205 - val_loss: 0.1734
Epoch 8/10
129/129 ━━━━━━━━━━━━━━━━━━━━ 477s 4s/step - accuracy: 0.9793 - loss: 0.0585 - val_accu

In [21]:
from sklearn.metrics import classification_report
import numpy as np

# Predict on the validation set
y_val_pred_prob = model.predict(X_val)

# Convert probabilities to class labels
y_val_pred = np.argmax(y_val_pred_prob, axis=1)
y_val_true = np.argmax(y_val, axis=1)

# Print classification report
print(classification_report(y_val_true, y_val_pred, target_names=class_names))

33/33 ━━━━━━━━━━━━━━━━━━━━ 98s 3s/step
              precision    recall  f1-score   support

          AD       1.00      0.99      0.99       244
          CI       0.99      1.00      0.99       511
          CN       0.99      0.99      0.99       276

    accuracy                           0.99      1031
   macro avg       0.99      0.99      0.99      1031
weighted avg       0.99      0.99      0.99      1031

